In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
import ipywidgets as widgets
from ipywidgets import interact
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib widget

In [14]:
fig, (ax1, ax2) = plt.subplots(2, 1)

x0 = 10 * (2 * np.random.rand(100) + 1)
ax1.scatter(x0, np.zeros_like(x0))

cell = 3.0
x1 = x0 - np.round(x0 / cell) * cell
ax2.scatter(x1, np.zeros_like(x1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
d1 = {1: 2}
d2 = d1
def f(d):
    d[2] = 3
f(d2)
d1

{1: 2, 2: 3}

# Data and plots

In [2]:
people = pd.read_csv('people.tab', sep='\s+')
n = people.shape[0]
people.head()

,age,weight,height,gender,married,number_of_kids,pet,expenses
0,25,61.7,121.12,other,False,2,ferret,23.442995
1,37,63.9,145.00,man,True,6,dog,96.836827
2,41,50.2,145.03,woman,True,2,hedgehog,312.676930
3,43,72.4,179.90,man,False,1,dog,447.428383
4,26,78.4,163.91,man,False,1,hedgehog,-78.227992


In [3]:
people.describe()

,age,weight,height,number_of_kids,expenses
count,500.000000,500.000000,500.00000,500.00000,500.000000
mean,39.484000,66.389000,168.18036,1.55800,478.598178
std,8.976352,12.947799,19.65701,1.38798,567.466205
min,17.000000,19.400000,113.59000,0.00000,-685.682537
25%,33.000000,57.600000,155.64500,0.75000,74.514280
50%,39.000000,66.600000,168.96500,1.00000,402.219536
75%,45.000000,75.300000,180.11500,2.00000,802.721954
max,72.000000,107.200000,235.25000,6.00000,3503.903462


There don't seem to be any errors in the dataset. As for the quantitative stats:
- there are 500 points;
- there are 4 continuous features (`age`, `weight`, `height` and `number_of_kids`);
- there are 3 noncontinuous features (boolean `married` and categorical `gender` and `pet`).

Let's look at the correlations between the quantitative features.

In [4]:
qual_vars = ['gender', 'married', 'pet']
for qual_var in qual_vars:
    people[qual_var] = people[qual_var].astype('str')
qual = people[qual_vars]

quant_vars = ['age', 'weight', 'height', 'number_of_kids', 'expenses']
quant = people[quant_vars]

cont_vars = ['age', 'weight', 'height', 'expenses']
cont = people[cont_vars]

sns.pairplot(quant, height=1, aspect=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

And now the correlation matrix.

In [5]:
fig1, ax1 = plt.subplots()
corr = people.corr()
sns.heatmap(people.corr(), ax=ax1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

So, for the most part, `expenses` is really only correlated with `age`.

Let's look at the boxplots.

In [6]:
fig2, ax2 = plt.subplots()
def boxplots(*args, **kwargs):
    col = kwargs['Column:']
    ax2.clear()
    sns.boxplot(x=col, data=people, ax=ax2)

interact(boxplots, **{'Column:': people.columns})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='Column:', options=('age', 'weight', 'height', 'gender', 'married',…

<function __main__.boxplots(*args, **kwargs)>

and the count/histplots.

In [7]:
fig3, ax3 = plt.subplots()
def barplots(*args, **kwargs):
    col = kwargs['Column:']
    ax3.clear()
    if col in quant_vars:
        sns.histplot(x=col, data=people, ax=ax3)
    else:
        sns.countplot(x=col, data=people, ax=ax3)

interact(barplots, **{'Column:': people.columns})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='Column:', options=('age', 'weight', 'height', 'gender', 'married',…

<function __main__.barplots(*args, **kwargs)>

# Hypothesis testing

## Hypothesis 1

$H_0: \mu_h = 170\text{ cm}$

$H_1: \mu_h < 170\text{ cm}$

We can do a standard $t$-test. Assumptions:
- $h$ is normally distributed: it is, in general.

In [8]:
h = people['height']
stats.ttest_1samp(h, popmean=170, alternative='less')

Ttest_1sampResult(statistic=-2.0699174047035447, pvalue=0.019487018511329824)

## Hypothesis 2

$H_0: \text{med}(h) = 165\text{ cm}$

$H_1: \text{med}(h) < 165\text{ cm}$

Here the most appropriate test is Wilcoxon signed-ranks test for samples $h$ and $Y = 165\text{ cm}$. Assumptions:
- ???

In [9]:
stats.wilcoxon(h, np.ones_like(h) * 165, alternative='less')

WilcoxonResult(statistic=74415.5, pvalue=0.9998676919653384)

# Confidence intervals
Here, `age` seems to be sort-of normally distributed, but in reality age is not distributed like it, so we shall not assume it. This, of course, complicates matters.

In [10]:
X = people['age']
gamma = 0.99
alpha = 1-gamma

## $\mu$
Since the sample size is fairly large, we could assume $\sqrt{n} (\bar{X} - \mu) / S \sim \mathcal{N}(0, 1)$, which would give us a confidence interval:
$$\left(\bar{X}-\frac{q(1-\alpha/2)}{\sqrt{n}}S, \bar{X}+\frac{q(1-\alpha/2)}{\sqrt{n}}S\right)$$

In [11]:
X_mean = np.mean(X)
X_var = np.sum((X - X_mean)**2)/(n-1)
X_sd = np.sqrt(X_var)
q = stats.norm().ppf(1-alpha/2)
mean_dev = q / np.sqrt(n) * X_sd
min_mean = X_mean - mean_dev
max_mean = X_mean + mean_dev
print(f'{100*gamma}% CI for mean: ({min_mean}, {max_mean})')

99.0% CI for mean: (38.44997283505816, 40.51802716494184)


## $\sigma$
There are a few ideas.

First is to assume (for a moment) that $X$ is normally distributed, and use the formula $nS^2/\sigma^2 \sim \chi^2(n-1)$, where we get confidence intervals:
$$\left(\frac{nS^2}{\chi^2(1-\alpha/2, n-1)}, \frac{nS^2}{\chi^2(\alpha/2, n-1)}\right)$$
for $\sigma^2$, which we may then simply take a square root of.

In [12]:
min1_sd = np.sqrt(n*X_var/stats.chi2(n-1).ppf(1-alpha/2))
max1_sd = np.sqrt(n*X_var/stats.chi2(n-1).ppf(alpha/2))
print(f'{100*gamma}% CI for sd (V1): ({min1_sd}, {max1_sd})')

99.0% CI for sd (V1): (8.304852688169143, 9.777895893016343)


`age`**Note**: Taken from taken from https://stats.stackexchange.com/questions/105337/asymptotic-distribution-of-sample-variance-of-non-normal-sample.

Another one is to use $\sqrt{n} (S^2-\sigma^2) \to_d \mathcal{N}(0, \mu_4 - \sigma^4)$, where $\mu_4$ is the fourth central moment $\mu_4 = E(X-\hat{X})^4$, which would give:
$$\left(S^2 - \frac{q(1-\alpha/2)}{\sqrt{n}}\sqrt{\hat{\mu_4}-S^4}, S^2 + \frac{q(1-\alpha/2)}{\sqrt{n}}\sqrt{\hat{\mu_4}-S^4}\right)$$
for $\sigma^2$, which we would take a square root of. We shall estimate $\hat{\mu_4}$ in a straightforward manner.

In [13]:
mu_4 = np.mean((X-X_mean)**4)
S2_dev = stats.norm().ppf(1-alpha/2)/np.sqrt(n) * np.sqrt(mu_4 - X_var**2)
min2_sd = np.sqrt(X_var-S2_dev)
max2_sd = np.sqrt(X_var+S2_dev)
print(f'{100*gamma}% CI for sd (V2): ({min2_sd}, {max2_sd})')

99.0% CI for sd (V2): (8.113220676523337, 9.763474680111814)


## Bayesian inference of $\mu$ and $\sigma$
We could also try to use Bayesian methods for deriving these intervals.

In [14]:
bayes_mu, _, bayes_sd = stats.bayes_mvs(X, alpha=gamma)
print(f'{100*gamma}% CI for mean (Bayes): {bayes_mu.minmax})')
print(f'{100*gamma}% CI for sd (Bayes): {bayes_sd.minmax}')

99.0% CI for mean (Bayes): (38.44600329675418, 40.521996703245826))
99.0% CI for sd (Bayes): (8.296543678897008, 9.768113103280314)


## Quantiles
**Note:** Taken from https://stats.stackexchange.com/questions/99829/how-to-obtain-a-confidence-interval-for-a-percentile

In [15]:
def quantileCI(q):
    d = int(np.sqrt(n))
    dist = stats.binom(n, q)
    lx = dist.ppf(alpha/2) + np.arange(-d, d)
    lx = lx[lx >= 0]
    ux = dist.ppf(1-alpha/2) + np.arange(-d, d)
    ux = ux[ux < n]
    
    lu_pairs = [(int(l), int(u)) for l in lx for u in ux
                if dist.cdf(u) - dist.cdf(l) >= 1-alpha]
    X_s = np.sort(X)
    l0, u0 = min(lu_pairs, key=lambda lu: X_s[lu[1]] - X_s[lu[0]])
    return (X_s[l0], X_s[u0])

for q in [0.25, 0.5, 0.75]:
    print(f'{100*gamma}% CI for quantile {q}: {quantileCI(q)}')

99.0% CI for quantile 0.25: (32, 35)
99.0% CI for quantile 0.5: (38, 40)
99.0% CI for quantile 0.75: (43, 47)


# Hypothesis testing II

## Hypothesis 1
We are supposed to pick one variable, but we could as well check all of them (aside from `gender`, of course)

$H_0$: means of values for men and women are equal

$H_1$: means of values for men and women are not equal

These can be checked using the $t$-test.

In [16]:
for quant_var in quant_vars:
    val_women = people[quant_var][people['gender'] == 'woman']
    val_men = people[quant_var][people['gender'] == 'man']
    stat, pv = stats.ttest_ind(val_women, val_men,
                               equal_var = False,
                               alternative='two-sided')
    print(f'Test for {quant_var}: t = {stat}, p = {pv}')

Test for age: t = 0.9173914514042579, p = 0.35942430946193793
Test for weight: t = -1.4298499464304124, p = 0.15346457455970736
Test for height: t = -1.3098950140520111, p = 0.19088863469241904
Test for number_of_kids: t = -0.4959202663374141, p = 0.620187530695445
Test for expenses: t = 0.4105633456676251, p = 0.6815843923892749


## Hypothesis 2

$H_0$: these (quantitative) variables are independent

$H_1$: these (quantitative) variables are dependent

Of course we shall check it for all pairs. We will use Spearman's $\rho$.

In [17]:
quant_pv = pd.DataFrame(columns=quant_vars, index=quant_vars, dtype=float)

for var1 in quant_vars:
    for var2 in quant_vars:
        pv = stats.spearmanr(people[var1], people[var2])[1]
        quant_pv.loc[var1, var2] = pv
        print(f'{var1} <-> {var2}: p = {pv}')

fig4, ax4 = plt.subplots()
sns.heatmap(quant_pv, ax=ax4)

age <-> age: p = 0.0
age <-> weight: p = 0.1538478459298146
age <-> height: p = 0.25452758457378305
age <-> number_of_kids: p = 0.8638567464685418
age <-> expenses: p = 2.1400774861481664e-146
weight <-> age: p = 0.1538478459298146
weight <-> weight: p = 0.0
weight <-> height: p = 9.520881755284377e-60
weight <-> number_of_kids: p = 0.48841727016347136
weight <-> expenses: p = 0.6600638422784821
height <-> age: p = 0.2545275845737831
height <-> weight: p = 9.52088175528492e-60
height <-> height: p = 0.0
height <-> number_of_kids: p = 0.5215269938776141
height <-> expenses: p = 0.6920268883759064
number_of_kids <-> age: p = 0.8638567464685418
number_of_kids <-> weight: p = 0.48841727016347136
number_of_kids <-> height: p = 0.5215269938776141
number_of_kids <-> number_of_kids: p = 0.0
number_of_kids <-> expenses: p = 0.2904480606280596
expenses <-> age: p = 2.1400774861476796e-146
expenses <-> weight: p = 0.6600638422784821
expenses <-> height: p = 0.6920268883759064
expenses <-> number_

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

## Hypothesis 3
Much the same thing but with qualitative variables. We will use Pearson's $\chi^2$ independence test.

In [18]:
qual_pv = pd.DataFrame(columns=qual_vars, index=qual_vars, dtype=float)

for var1 in qual_vars:
    for var2 in qual_vars:
        counts = pd.crosstab(people[var1], people[var2])
        pv = stats.contingency.chi2_contingency(counts)[1]
        qual_pv.loc[var1, var2] = pv
        print(f'{var1} <-> {var2}: p = {pv}')

fig5, ax5 = plt.subplots()
sns.heatmap(qual_pv, ax=ax5)

gender <-> gender: p = 3.5694127797773694e-215
gender <-> married: p = 0.2729287015896417
gender <-> pet: p = 0.8325018435693856
married <-> gender: p = 0.2729287015896416
married <-> married: p = 8.65741727035078e-110
married <-> pet: p = 0.21403498460590317
pet <-> gender: p = 0.8325018435693855
pet <-> married: p = 0.21403498460590317
pet <-> pet: p = 0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

## Hypothesis 4
We may wish to check whether `age` is indeed normally distributed, using the Shapiro-Wilk test.

In [19]:
stats.shapiro(people['age'])

ShapiroResult(statistic=0.9817890524864197, pvalue=6.5907856878766324e-06)